#  Canada's neighbourhoods


This is a notebook to get data Canada's neighbourhood, based on the wikipedia entry https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
then the neighbourhood will be clustered to find neighbourhoods of the same kind:

## 1.  Preprocessing: getting and cleaning the data

In [1]:
import numpy as np
import pandas as pd
import pandas as pd
from bs4 import BeautifulSoup
import requests

Getting the data and stroing them into a list:

In [107]:
#get data of table and create list
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('tbody')
#table.text
l = []
for entry in table.find_all('tr'):
    a = entry.text.split('\n')
    l.append(a[1:4])


creating a panda frame:

In [108]:
#make panda frame
df0 = pd.DataFrame(l[1:],columns = l[0])
df0.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignore cells with a borough that is Not assigned and replace  neighbourhood 'not assigned'  by borough name

In [187]:

#drop 'not assigned borough'
df = df0[df0.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)

# neighbourhood 'not assigned' and replace by borough name
tt = df[df.Neighbourhood == 'Not assigned']
for num in tt.index:
    df.loc[num].Neighbourhood =  df.loc[num].Borough
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


 The rows with smame Postvode will be combined into one row 
    with the neighborhoods separated with a comma.

In [178]:
#groupy by postcode andcreate data framw

bourug = df.groupby(['Postcode']).Borough.unique().to_frame().reset_index()
neigh = df.groupby('Postcode').Neighbourhood.unique().to_frame().reset_index()
df_all = bourug.merge(neigh, on='Postcode', how='left')

#df_all = bourug.set_index("Postcode").join(neigh.set_index("Postcode"))
#neigh.head()
df_all.head()

,Postcode,Borough,Neighbourhood
0,M1B,[Scarborough],"[Rouge, Malvern]"
1,M1C,[Scarborough],"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,[Scarborough],"[Guildwood, Morningside, West Hill]"
3,M1G,[Scarborough],[Woburn]
4,M1H,[Scarborough],[Cedarbrae]


make data frame prettier/ so get rid off "[ ]"

In [174]:
#func gives back sting
def f_strings(t):
    L = t
    makeitastring = ', '.join(map(str, L))
    return(makeitastring)

In [185]:

df_all.Borough = df_all.Borough.apply(f_strings)
df_all.Neighbourhood = df_all.Neighbourhood.apply(f_strings)


thus the Fincal data frame is:

In [186]:
df_all.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [192]:
row,col = df_all.shape
print(' the data frame has ', row, 'rows')

 the data frame has  103 rows
